# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [4]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
    "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
    "uses statistical modeling and machine learning "
    "to provide crucial insights. With a knack for data, "
    "the Data Analyst Agent is the cornerstone for "
    "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

In [8]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
    "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
    "markets and quantitative analysis, this agent "
    "devises and refines trading strategies. It evaluates "
    "the performance of different approaches to determine "
    "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

In [11]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
    "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
    "and logistical details of potential trades. By evaluating "
    "these factors, it provides well-founded suggestions for "
    "when and how trades should be executed to maximize "
    "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

In [12]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
    "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
    "and market dynamics, this agent scrutinizes the potential "
    "risks of proposed trades. It offers a detailed analysis of "
    "risk exposure and suggests safeguards to ensure that "
    "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

## Creating Tasks

In [14]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [16]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [17]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [20]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [21]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[
        data_analyst_agent,
        trading_strategy_agent,
        execution_agent,
        risk_management_agent,
    ],
    tasks=[
        data_analysis_task,
        strategy_development_task,
        execution_planning_task,
        risk_assessment_task,
    ],
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7),
    process=Process.hierarchical,
    verbose=True,
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [22]:
# Example data for kicking off the process
financial_trading_inputs = {
    "stock_selection": "AAPL",
    "initial_capital": "100000",
    "risk_tolerance": "Medium",
    "trading_strategy_preference": "Day Trading",
    "news_impact_consideration": True,
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [23]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


# Agent: Crew Manager
## Thought: I need to gather relevant information about the stock AAPL and analyze market data to identify trends and predict market movements in order to provide insights and alerts about significant market opportunities or threats for AAPL.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.apple.com/investor/\"}"
## Tool Output: 

Investor Relations - Apple
Skip to main content
Exit
Apple
Store
Mac
iPad
iPhone
Watch
AirPods
TV &
Home
Entertainment
Accessories
Support
0 +
Investor Relations
Stock Price SEC Filings Leadership and Governance Our Values FAQ Contact
Investor Relations
Investor Updates
2025 Proxy Materials
View the Proxy Statement
View the Form
10-K
2025 Annual Meeting of Shareholders
Apple will host the 2025 Annual 

- Display the final result as Markdown.

In [24]:
from IPython.display import Markdown

Markdown(result.raw)

Apple Inc. (AAPL) presents a unique investment opportunity, but like any investment, it comes with its own set of risks that must be carefully analyzed and mitigated. Here is a detailed risk analysis and suggested mitigation strategies for AAPL trading based on insights gathered:

1. **Market Volatility**: AAPL's stock price has been subject to significant fluctuations influenced by broader market trends, technological sector shifts, and macroeconomic factors. For instance, market forecasts for AAPL in 2025 suggest a price range between $250 and $276.
   
   *Mitigation Strategy*: Implement stop-loss orders to protect against severe downside risk and consider using options strategies such as protective puts to hedge against volatility.

2. **Earnings and Revenue Forecasts**: Analysts predict that Apple could reach $415 billion in revenue for fiscal 2025. This growth forecast is contingent on successful product launches and market expansion.
   
   *Mitigation Strategy*: Continuously monitor earnings reports and analyst updates to adjust positions accordingly. Diversify the portfolio to include other technology and consumer electronics stocks which might perform well if Apple's growth slows.

3. **Sector Risks**: As part of the technology sector, Apple is exposed to rapid technological changes and competitive pressures from firms like Samsung and Google.
   
   *Mitigation Strategy*: Consider diversifying into other sectors or ETFs that can balance exposure to technology risks.

4. **Economic Factors**: Global economic conditions can rapidly change, affecting consumer purchasing power and Apple's international revenues.
   
   *Mitigation Strategy*: Utilize a balanced approach by investing in stocks from different regions, or consider currency-hedged funds if international exposure is significant.

5. **Regulatory Risks**: Apple's operations are subject to global regulations which can impact its business strategies, especially regarding data privacy and antitrust laws.
   
   *Mitigation Strategy*: Keep abreast of regulatory changes and consider reducing exposure if regulatory pressures increase significantly, impacting Apple's business model.

In summary, while AAPL remains a strong investment with robust growth potential, especially with a projected market cap maintaining above $3 trillion, its risks must be managed with strategic diversification, portfolio hedging, and staying informed about market and industry developments. These mitigation strategies aim to protect the firm's investment in AAPL while aligning with its overall risk tolerance.